# Live games

## Housekeeping

### import dependendencies

In [ ]:
import datetime as dt
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from hockey_rink import NHLRink

import chickenstats.utilities
from chickenstats.chicken_nhl import Scraper, Season
from chickenstats.chicken_nhl.helpers import norm_coords
from chickenstats.chicken_nhl.info import NHL_COLORS

### Pandas options

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Environment variables

In [ ]:
env_path = Path("../.env")
load_dotenv(env_path)

### Chickenstats matplotlib style

In [ ]:
plt.style.use("chickenstats")

## Scrape data

### Schedule and game IDs

In [ ]:
season = Season(2024)

In [ ]:
schedule = season.schedule()

In [ ]:
team = "NSH"
game_state = "LIVE"

live_game_conds = np.logical_and(
    np.logical_or(schedule.home_team == team, schedule.away_team == team),
    np.logical_or(schedule.game_state == game_state, schedule.game_id == 2024020211),
)

live_game_id = schedule.loc[schedule.game_id == 2024020211].game_id.iloc[0]

In [ ]:
schedule

### Play-by-play

In [ ]:
scraper = Scraper(live_game_id)

In [ ]:
pbp = scraper.play_by_play

In [ ]:
stats = scraper.stats.copy(deep=True)

In [ ]:
forwards = scraper.lines.copy(deep=True)

In [ ]:
scraper.prep_lines(position="d")
defense = scraper.lines.copy(deep=True)

In [ ]:
conds = np.logical_and(defense.strength_state == "5v5", defense.team == "NSH")
defense.loc[conds].sort_values(by="toi", ascending=False).head(10)

In [ ]:
conds = np.logical_and(forwards.strength_state == "5v5", forwards.team == "NSH")
forwards.loc[conds].sort_values(by="xga_p60", ascending=False).head(10)

In [ ]:
lines = forwards.copy()

In [ ]:
# Setting single team and other filter conditions
team = "NSH"
strength_state = "5v5"
toi_min = 0.5

# Setting filter conditions and filtering data
conds = np.logical_and(lines.strength_state == strength_state, lines.toi >= toi_min)
plot_lines = lines.loc[conds].sort_values(by="xgf_percent", ascending=False).reset_index(drop=True)

# Setting overall figures
fig, ax = plt.subplots(dpi=650, figsize=(8, 5))

# Aesthetics, likes the tight tight layout and despining axes
fig.tight_layout()
sns.despine()

# Getting the averages and drawing the average lines
xga_mean = plot_lines.xga_p60.mean()
xgf_mean = plot_lines.xgf_p60.mean()

ax.axvline(x=xga_mean, zorder=-1, alpha=0.5)
ax.axhline(y=xgf_mean, zorder=-1, alpha=0.5)

# Setting the size norm so bubbles are consistent across figures
size_norm = (plot_lines.toi.min(), plot_lines.toi.max())

# Getting plot colors based on team
colors = NHL_COLORS[team]

# Filtering data and plotting the non-selected teams first
conds = plot_lines.team != team
plot_data = plot_lines.loc[conds]

# They all get gray colors
facecolor = colors["MISS"]
edgecolor = colors["MISS"]

# Plotting the non-selected teams' data
sns.scatterplot(
    data=plot_data,
    x="xga_p60",
    y="xgf_p60",
    size="toi",
    sizes=(20, 150),
    size_norm=size_norm,
    lw=1.5,
    facecolor=facecolor,
    edgecolor=edgecolor,
    alpha=0.5,
    legend=True,
)

# Filtering the data and plotting the selected team
conds = plot_lines.team == team
plot_data = plot_lines.loc[conds]

# Setting the colors
facecolor = colors["GOAL"]
edgecolor = colors["SHOT"]

# Plotting the selected teams' data
sns.scatterplot(
    data=plot_data,
    x="xga_p60",
    y="xgf_p60",
    size="toi",
    sizes=(20, 150),
    lw=1.5,
    facecolor=facecolor,
    edgecolor=edgecolor,
    alpha=0.8,
    legend=False,
)

# Iterating through the dataframe to label the bubbles
for _row, line in plot_data.iterrows():
    # Setting x and y positions that are slightly offset from the data they point to
    x_position = line.xga_p60 + 0.25
    y_position = line.xgf_p60 - 0.25

    # Annotation options
    arrow_props = {"arrowstyle": "simple", "linewidth": 0.25, "color": "tab:gray"}

    # Plotting the annotation
    ax.annotate(
        text=f"{line.forwards}",
        xy=(line.xga_p60, line.xgf_p60),
        xytext=(x_position, y_position),
        fontsize=6,
        bbox={"facecolor": "white", "alpha": 0.5, "edgecolor": "white", "pad": 0},
        arrowprops=arrow_props,
    )

# Setting axis lables
ax.axes.set_xlabel("xGA per 60 minutes")
ax.axes.set_ylabel("xGF per 60 minutes")

# Setting figure suptitle and subtitle
fig_suptitle = "Nashville Predators forwards are generating 5v5 offense at rates above the NHL average"
fig.suptitle(fig_suptitle, x=0.01, y=1.08, fontsize=11, fontweight="bold", horizontalalignment="left")

todays_date = dt.datetime.now().strftime("%Y-%m-%d")
subtitle = f"NHL forward line combinations | >{toi_min} min. TOI at 5v5 | 2024-25 season, as of {todays_date}"
fig.text(s=subtitle, x=0.01, y=1.02, fontsize=10, horizontalalignment="left")

# Attribution
attribution = "Data & xG model @chickenandstats | Viz @chickenandstats"
fig.text(s=attribution, x=0.99, y=-0.05, fontsize=8, horizontalalignment="right", style="italic")

# Save figure
savepath = Path(f"./charts/5v5_xgf_xga_{team}.png")
# fig.savefig(savepath, transparent=False, bbox_inches="tight")